## Implementamos un baseline basado en las últimas vistas de cada usuario

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import random
from datetime import datetime

In [2]:
train_file = 'train_dataset.jl'
item_file = 'item_data.jl'

train_data = pd.read_json(train_file, lines=True, nrows=3000, orient='columns')
#item_data = pd.read_json(item_file, lines=True, orient='columns')

In [3]:
train_data.head(2)

,user_history,item_bought
0,"[{'event_info': 1786148, 'event_timestamp': '2...",1748830
1,"[{'event_info': 643652, 'event_timestamp': '20...",228737


In [4]:
# Esta función sirve para acomodar el archivo json en algo tabular

def preprocess_hist(df):
    df['user_view']= pd.Series(dtype='object')
    df['timestamps']= pd.Series(dtype='object')
    df['user_search']= pd.Series(dtype='object')
    df['search_timestamps']= pd.Series(dtype='object')

    for i in df.index:
        lista_view=[]
        lista_time=[]
        lista_search=[]
        lista_search_t=[]
        for item in df.user_history[i]:
            if item['event_type'] =='view':
                lista_view.append(int(item['event_info']))
                time_string=item['event_timestamp'].replace("T", " ").split('.')[0]
                timestamp=datetime.timestamp(datetime.strptime(time_string, '%Y-%m-%d %H:%M:%S'))
                lista_time.append(int(timestamp))
            if item['event_type'] =='search':
                lista_search.append(item['event_info'])
                time_string=item['event_timestamp'].replace("T", " ").split('.')[0]
                timestamp=datetime.timestamp(datetime.strptime(time_string, '%Y-%m-%d %H:%M:%S'))
                lista_search_t.append(int(timestamp))

        df.at[i,'user_view']= lista_view
        df.at[i,'timestamps']= lista_time

        df.at[i,'user_search']= lista_search
        df.at[i,'search_timestamps']= lista_search_t
    return df

In [5]:
# Apligo la función preprocess_hist a la train part
train_data = preprocess_hist(train_data)
train_data.drop('user_history', axis=1, inplace=True)
train_data.head(4)

,item_bought,user_view,timestamps,user_search,search_timestamps
0,1748830,"[1786148, 1786148, 1615991, 1615991, 1615991, ...","[1571495142, 1571495157, 1571495246, 157149531...",[RELOGIO SMARTWATCH],[1571495167]
1,228737,"[643652, 1156086, 1943604, 206667, 1282813, 22...","[1570395773, 1570452367, 1570452473, 157048520...","[DESMAMADEIRA ELETRICA, DESMAMADEIRA ELETRICA,...","[1570452329, 1570452377, 1570452379, 157048520..."
2,1909110,"[248595, 248595]","[1569944763, 1569946910]",[],[]
3,1197370,"[505541, 505541, 505541, 505541, 505541, 12300...","[1570562646, 1570562756, 1570562775, 157056280...","[RADIOBOSS, RADIOBOSS, SOUND FORGE, SOUND FORG...","[1570562631, 1570584752, 1570728776, 157088525..."


#### Importamos los metadatos

In [6]:
def jl_to_list(fname):
    output = []
    with open(fname) as f:
        for line in f:
            output.append(json.loads(line))
    return output


item_data = jl_to_list('item_data.jl')
metadata = {x['item_id']:x for x in item_data}
all_items = list(metadata.keys())
#metadata

#### Definimos la métrica que usaremos

In [7]:
def ndcg(y_pred, y_true):
    'Basada en el video de la presentación del challenge'
    dcg = 0
    idcg = 22.42461597 * len(y_true)
    for pred, true in zip(y_pred, y_true):
        position = 1
        for item in pred:
            if item == true:
                dcg += 12 / (np.log(1 + position))
            elif metadata[item]['domain_id'] == metadata[true]['domain_id']:
                dcg += 1 / (np.log(1 + position))
            position += 1
    score = dcg / idcg
    
    return score

#### Implementación del baseline basado en los últimos views de un usuario

Un usuario entra a la plataforma, mira 7 artículos (algunos o todos podrían estar repetidos) y finalmente compra uno. La idea es recomendarle 10 artículos en su próxima visita a la plataforma. Cómo lo hacemos?

El baseline simplemente se basa en recomendarle, en su proxima visita, esos mismos 7 artículos que miró antes (descartando los que estuvieran repetidos) más otros 3 artículos elegidos aleatoriamente (o la cantidad necesaria hasta completar 10 artículos). No ordenamos los artículos recomendados por "últimos artículos vistos" y los artículos elegidos aleatoriamente son elegidos dentro de todo el data set (no sólo dentro de la categoría a la cual pertenece el item comprado). En caso de que el usuario haya visto más de 10 artículos simplemente nos quedamos con 10 de ellos en el orden en que fueron cargados al dataset.

La selección aleatoria la hacemos siempre con el mismo seed para procurar la reproducibilidad del experimento.

In [8]:
def baseline_row(row,df,n_items=10):
    'Hacemos la recomendación para cada item comprado (row)'
    viewed = list(dict.fromkeys(df["user_view"][row])) # descartamos los views repetidos
    
    if len(viewed) == n_items:
        return viewed
    elif len(viewed) > n_items:
        return viewed[:n_items]
    else:
        random.seed(123)
        return viewed + random.choices(list(train_data["item_bought"]),k=n_items-len(viewed))
    baseline(2,train_data)

In [9]:
from sklearn.model_selection import train_test_split
X, y = train_data.drop(columns=["item_bought"]), train_data["item_bought"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [10]:
def baseline():
    'Recomendación para todos los items comprados'
    y_pred = []
    for row in range(len(X_test)):
        recom = baseline_row(row,X_test)
        y_pred.append(recom)
    return y_pred    

y_pred = baseline()

#### Veamos el score

In [11]:
score = ndcg(y_pred,y_test)
print('Score:', score)

Score: 0.16049504552784666
